In [1]:
from rdkit import Chem

In [2]:
from rassp.msutil.masscompute import FragmentFormulaPeakEnumerator

#valid_atoms = {1, 5, 6, 7, 8, 9, 14, 15, 16, 17, 35, 53}
valid_atoms = {1, 6, 7, 8, 9, 15, 16, 17}
num_peaks_per_formula = 12
#max_formulae = 100000
max_formulae = 4096

ffe = FragmentFormulaPeakEnumerator(sorted(valid_atoms), use_highres=True, max_peak_num=num_peaks_per_formula)

def goodmol(mol):
        if len(mol.GetAtoms()) > 48:
            return False
        
        atoms = { a.GetAtomicNum() for a in mol.GetAtoms() }
        if not atoms < valid_atoms:
            return False
            
        f,m = ffe.get_frag_formulae(mol)
        if len(f) > max_formulae:
            return False

        return True


In [10]:
%%time
smi = 'zinc/small.smi'
smallsmi = []

with open(smi) as sf:
    smallsmi = [ s for s in sf if goodmol(Chem.AddHs(Chem.MolFromSmiles(s))) ]



CPU times: user 9.95 s, sys: 31.6 ms, total: 9.98 s
Wall time: 9.98 s


In [9]:
len(smallsmi)

470

In [13]:
import multiprocessing

try:
    cpus = multiprocessing.cpu_count()
except NotImplementedError:
    cpus = 2   # arbitrary default

cpus //= 2 # ht
cpus

12

In [ ]:
%%time

smi='zinc/30M.smi'
pool = multiprocessing.Pool(processes=cpus)

def goodsmiles(s):
    return goodmol(Chem.AddHs(Chem.MolFromSmiles(s)))

with open(smi) as sf:
    sfs = list(sf)
    
good = pool.map(goodsmiles, sfs)

smallsmi = [ x[0] for x in zip(sfs,good) if x[1] ]

[14:30:38] Explicit valence for atom # 22 F, 2, is greater than permitted
[14:51:18] Explicit valence for atom # 25 O, 3, is greater than permitted
[15:05:47] Explicit valence for atom # 20 O, 3, is greater than permitted
[15:15:32] Explicit valence for atom # 29 O, 3, is greater than permitted
[15:26:53] Explicit valence for atom # 22 Cl, 2, is greater than permitted
[15:31:19] SMILES Parse Error: unclosed ring for input: 'O=C1CCC(CN1)c1nnc(n1CC1CCCCO1)N1CCC(CC(NC(=O)c1cnn(c1)C(C)(C)C)CNC(=O)OC(C)(C)C)C
'
[15:34:02] Explicit valence for atom # 24 O, 3, is greater than permitted
[15:42:26] Explicit valence for atom # 18 F, 2, is greater than permitted


In [ ]:
len(smallsmi)